Initialisation script (run first and see required libraries):

In [1]:
# Required libraries: numpy, sympy, matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from IPython.display import display, Math

sp.init_printing(use_latex=True)

### Normality check
Check if a matrix is normal (change line 4 to check other properties)

This checks that $A\cdot A^* = A^*\cdot A$.</br>
This script checks that a matrix is normal for a given set of values.</br>
To check just one matrix, ignore the variable 't'.

In [2]:
# Which value t is the matrix normal: A A* = A* A
I = np.sqrt(-1) # in case you need :)
def is_normal(matrix):
    return np.allclose(matrix @ matrix.conj().T, matrix.conj().T @ matrix)

# List of t values to check
t_values = [0, 1, 2, 3, 4, 5, 6]

# Check normality for each t
for t in t_values:
    A = np.array([
        [1, 2, 3, 4],
        [4, 1, 2, 3],
        [3, 4, 1, 2],
        [t, 3, 4, 1]
    ], dtype=complex)

    if is_normal(A):
        print(f"Matrix is normal for t = {t}")
    else:
        print(f"Matrix is NOT normal for t = {t}")

Matrix is NOT normal for t = 0
Matrix is NOT normal for t = 1
Matrix is normal for t = 2
Matrix is NOT normal for t = 3
Matrix is NOT normal for t = 4
Matrix is NOT normal for t = 5
Matrix is NOT normal for t = 6


### Eigenvalues and eigenvectors from matrix
Enter a matrix, and it will give all eigenvalues, along with their corresponding eigenvector.</br>
The orthogonal matrix is also provided. This corresponds to only the eigenvalues in the given order.

In [13]:
# Enter your matrix
A = sp.Matrix([[1, 1, -4],
               [1, 4, -1],
               [-4, -1, 1]])

eigen = A.eigenvects()
eigenvalues = [ev[0] for ev in eigen]
eigenvectors = [ev[2][0].normalized() for ev in eigen]
for i in range(len(eigen)):
    display(Math(f"\\lambda_{i+1}:~~~" + sp.latex(eigenvalues[i]) + "~~,~" + sp.latex(eigenvectors[i])))

# Orthogonal change-of-basis matrix Q
Q = sp.Matrix.hstack(*eigenvectors)

print("\nOrthogonal matrix Q:")
display(Q)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


Orthogonal matrix Q:


⎡√2  -√6   -√3 ⎤
⎢──  ────  ────⎥
⎢2    6     3  ⎥
⎢              ⎥
⎢     √6   -√3 ⎥
⎢0    ──   ────⎥
⎢     3     3  ⎥
⎢              ⎥
⎢√2   √6    √3 ⎥
⎢──   ──    ── ⎥
⎣2    6     3  ⎦

### Gradient method
Use the gradient method (gradient descent) to find the smallest possible value of a multivariable function.</br>
It iterates over a function, moving in steps in the opposite direction of the gradient to find the minimum value.

In [127]:
# Define symbols
x1, x2, x3 = sp.symbols('x1 x2 x3')
x = sp.Matrix([x1, x2, x3])

# Inputs
q = 5*x1**2 + 5*x2**2 + 8*x3**2 + 8*x1*x2 - 4*x1*x3 + 4*x2*x3 - 22*x1 - 32*x2 - 20*x3 + 53 # Function
a = 0.02 # Step size (aka learning rate)
x0 = np.array([2, 2, 2]) # Starting point (initial guess)
target = 10 # Number of iterations to perform

# Compute the gradient symbolically
grad_q = sp.Matrix([sp.diff(q, var) for var in x])

# Convert symbolic gradient to a numerical function
grad_q_func = sp.lambdify((x1, x2, x3), grad_q, 'numpy')

# Perform iterations
for _ in range(target):
    grad_val = np.array(grad_q_func(*x0)).astype(float).flatten()
    x0 = x0 - a * grad_val

display(Math(f"x_{{{target}}} ={sp.Matrix(x0)._repr_latex_().replace('$', '')}"))

<IPython.core.display.Math object>

### Determine stationary points
Input a list of points and a function to get whether each point is stationary and what type.

In [137]:
# Define symbols
x1, x2, x3 = sp.symbols('x1 x2 x3')
x = sp.Matrix([x1, x2, x3])
# Inputs
q = 5*x1**2 + 5*x2**2 + 8*x3**2 + 8*x1*x2 - 4*x1*x3 + 4*x2*x3 - 22*x1 - 32*x2 - 20*x3 + 53
points = [(-1,4,0),(2,-2,1),(1,2,1),(-3,6,-1),(3,0,2),(-19,16,10)]

# Compute the gradient symbolically
grad_q = sp.Matrix([sp.diff(q, var) for var in x])
hessian_q = grad_q.jacobian(x)

# for each point, determine if stationary
table = "\\begin{array}{c|rl}"
for point in points:
    grad_at_point = grad_q.subs({x1: point[0], x2: point[1], x3: point[2]})
    if all(val == 0 for val in grad_at_point):
        table += f"{point}& \\text{{stationary}}&"
        # Evaluate Hessian at the point
        hess_val = hessian_q.subs({x1: point[0], x2: point[1], x3: point[2]})
        hess_val_np = np.array(hess_val.tolist(), dtype=np.float64)
        eigvals = np.linalg.eigvals(hess_val_np)

        if np.all(eigvals > 0):
            table += "\\text{→~~~~Local minimum}\\\\"
        elif np.all(eigvals < 0):
            table += "\\text{→~~~~Local maximum}\\\\"
        else:
            table += "\\text{→~~~~Saddle point}\\\\"
    else:
        table += f"{point} & \\text{{NOT stationary}}\\\\"
table+="\\end{array}"
display(Math(table.replace('$','')))

<IPython.core.display.Math object>

### Find Jacobian matrix and determinant
Enter a vector function to get the $\text{Jac}$ and $\det|\text{Jac}|$

In [5]:
# Define symbols
u, v, w = sp.symbols('u v w')
# Inputs
vector = sp.Matrix([u*sp.cos(v),
                    u*sp.sin(v)])

Jac_matrix = vector.jacobian(sp.Matrix([u, v]))

print("Jacobian matrix:")
display(Jac_matrix)
print("\nJacobian determinant:")
display(sp.det(Jac_matrix))

Jacobian matrix:


⎡cos(v)  -u⋅sin(v)⎤
⎢                 ⎥
⎣sin(v)  u⋅cos(v) ⎦


Jacobian determinant:


### Find Hessian matrix and determinant at a point
Enter a function and a specific point (optional) to get the Hessian matrix.

In [6]:
x, y = sp.symbols('x y')

# Inputs
f = 2*x**2 + y**2 - x*y**2
point = (3,5)

# Hessian matrix
H = sp.hessian(f, (x, y))

# Evaluate Hessian at a point
H_point = H.subs({x: point[0], y: point[1]})

# Determinant
det_H = H_point.det()

print(f"Hessian matrix =")
display(H)
print(f"\nHessian matrix at {point} =")
display(H_point)
print(f"\nDeterminant of H at {point} = {det_H}")

Hessian matrix =


⎡ 4     -2⋅y  ⎤
⎢             ⎥
⎣-2⋅y  2 - 2⋅x⎦


Hessian matrix at (3, 5) =


⎡ 4   -10⎤
⎢        ⎥
⎣-10  -4 ⎦


Determinant of H at (3, 5) = -116


### Riemann Integral

In [7]:
# Define variables
x,y,z = sp.symbols('x y z')
u,v,w = sp.symbols('u v w')

# Define vector field V
V1 = x + sp.exp(y*z)
V2 = 2*y - sp.exp(x*z)
V3 = 3*z + sp.exp(x*y)

# Compute divergence f = div(V)
f = sp.diff(V1, x) + sp.diff(V2, y) + sp.diff(V3, z)
f = sp.simplify(f)

# Parametrization r(u,v,w)
x_r = v * u**2 * sp.cos(w)
y_r = v * u**2 * sp.sin(w)
z_r = u

# Create r vector
r = sp.Matrix([x_r, y_r, z_r])

# Compute Jacobian matrix: partial derivatives of r wrt u, v, w
J = r.jacobian([u, v, w])
Jacobian_det = sp.simplify(J.det())

# Substitute x, y, z with parametric expressions in f
f_subs = f.subs({x: x_r, y: y_r, z: z_r})

# Full integrand: f(r(u,v,w)) * |det(Jacobian)|
integrand = sp.simplify(f_subs * sp.Abs(Jacobian_det))

# Perform the triple integral
I = sp.integrate(integrand, (u, 0, 1), (v, 0, 1), (w, 0, sp.pi/2))

# Output result
display(Math("\\text{Divergence}~~f(x, y, z) = " + sp.latex(f)))
display(Math("\\text{Jacobian determinant} = " + sp.latex(Jacobian_det)))
display(Math("\\text{Triple integral result} =" + sp.latex(I)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find function limit
Insert a function and a limit to get the value of the function at the limit.

In [11]:
# Define the symbol and the piecewise function
x,y = sp.symbols('x y')

# Inputs
f = (sp.cos(2*y)*(x**2-2*sp.sin(3*x)))/(5*x)
lim = 0

f_limit = sp.limit(f, x, lim)
display(Math("\\lim_{x\\to" + str(lim) + "}" + sp.latex(f) + "=" + sp.latex(f_limit)))

<IPython.core.display.Math object>

### Taylor polynomial
Works for 1, 2, or 3 variables. If only one or two variables are needed, just ignore them (leave variables as 0 on lines 6 and 7).</br>
The below example uses 2 variables $(x,y)$. e.g. $z$ is left as 0.

In [19]:
x, y, z = sp.symbols('x y z')

# Inputs
f = sp.exp(x)*sp.cos(y) # Function
order = 3 # Highest degree
x0 = (0,0,0) # Expansion point
new_point = (1,-1,0) # Point to approximate value of (optional)

approximation = f.series(x, x0[0], order+1).removeO().series(y, x0[1], order+1).removeO().series(z, x0[2], order+1).removeO()

display(Math("f(x,y,z)=" + sp.latex(f)))
display(Math("P_{f,x_0}(x,y,z)=" + sp.latex(approximation, order='rev-lex')))
display(Math("P_{f,x_0}" + str(new_point) + "=" + sp.latex(approximation.subs({x:new_point[0], y:new_point[1], z:new_point[2]}))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Orthogonalise vectors (Gram-Schmidt)
Input a list of vectors (line 21) to get the corresponding orthogonal and orthonormal set of vectors.</br>
This will output a list of vectors in the form:</br>
$\text{Vector}~\to~\text{Orthogonal vector}~\to~\text{Orthonormal vector}$</br>
An orthonormal vector is both orthogonal and normal (has length of 1).</br>
If your inputs do not change after being orthogonalised, they already form an orthogonal basis.

In [64]:
def gram_schmidt(vectors):
    ortho_vectors = []
    ortho_normal_vectors = []
    for v in vectors:
        w = v
        for u in ortho_vectors:
            proj = (u.dot(w) / u.dot(u)) * u
            w = w - proj
        ortho_vectors.append(sp.simplify(w))

        norm = sp.sqrt(w.dot(w))
        if norm != 0:
            normalized = sp.simplify(w / norm)
        else:
            normalized = w  # Handle zero vector case
        ortho_normal_vectors.append(normalized)
    return ortho_vectors, ortho_normal_vectors

# Define any number of column vectors
input_vectors = [sp.Matrix([1, 3, 0]),
                 sp.Matrix([6, -2, 1]),
                 sp.Matrix([3, -1, -20])]

# Apply Gram-Schmidt
orthogonal, orthonormal = gram_schmidt(input_vectors)

# Print results
for i, vec in enumerate(orthogonal, 1):
    display(Math(f"{sp.latex(input_vectors[i-1])} ~~\\to~~ {sp.latex(vec)} ~~\\to~~ {sp.latex(orthonormal[i-1])}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>